In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2

import numpy as np
import sklearn as sk
import pickle

Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_3'
batch_size = 16

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [6]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(10168, 10000, 1)
(7071, 10000, 1)
16 20


In [7]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"))

CPU times: user 5.38 s, sys: 653 ms, total: 6.04 s
Wall time: 6.04 s


In [8]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [11]:
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=32, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_2 (MonteCarl (None, 10000, 32)         544       
_________________________________________________________________
flatten_2 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                6400020   
Total params: 6,400,564
Trainable params: 6,400,564
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 96s - loss: 1.6700 - acc: 0.6001 - val_loss: 1.1846 - val_acc: 0.6884
Epoch 2/20
10168/10168 [==============================] - 96s - loss: 0.3180 - acc: 0.9303 - val_loss: 1.1559 - val_acc: 0.7003
Epoch 3/20
10168/10168 [==============================] - 96s - loss: 0.0944 - acc: 0.9857 - val_loss: 1.2399 - val_acc: 0.6969
Epoch 4/20
10168/10168 [==============================] - 96s - loss: 0.0389 - acc: 0.9949 - val_loss: 1.3582 - val_acc: 0.6880
Epoch 5/20
10168/10168 [==============================] - 96s - loss: 0.0211 - acc: 0.9978 - val_loss: 1.3974 - val_acc: 0.6952
Test loss: 1.39736776766
Test accuracy: 0.695234054353


In [ ]:
%%bash
tail mcNet_top10k_temptative_5_log.csv